# Project 4
## Students:
 > [Tanner Mengel]
 > [Ian Cox]
 
 

In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import re

In [9]:
print(tf.__version__)# you may want to upgrade to 2.10.0 

2.12.0


### Please Use Markdown
> for markdown, see here: https://www.ibm.com/docs/en/watson-studio-local/1.2.3?topic=notebooks-markdown-jupyter-cheatsheet

## Task 1

In [10]:
class TransformerModel():
    def __init__(self, vocab_size, embed_dim=256, num_heads=2, num_blocks=1, ff_dim=256, maxlen=80, rate=0.1):
        #initailize variables
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.num_blocks = num_blocks
        self.ff_dim = ff_dim
        self.maxlen = maxlen
        self.rate = rate

    def TransformerBlock(self, inputs):
        #create the transformer block as discribed in the writeup, use the Keras functional API (https://keras.io/guides/functional_api/)
        #MultiHeadAttention layer, specifiy 'use_causal_mask=True' (https://keras.io/api/layers/attention_layers/multi_head_attention/)
        MHA_Layer = layers.MultiHeadAttention(num_heads=self.num_heads,  key_dim=self.embed_dim,  dropout=self.rate)(inputs, inputs, use_causal_mask=True)
        
        #Dropout layer
        dropout_1 = layers.Dropout(rate=self.rate)(MHA_Layer)
        #LayerNormalization layer, specifiy 'epsilon=1e-6' (https://keras.io/api/layers/normalization_layers/layer_normalization/)
        normlayer_1 = layers.LayerNormalization(epsilon=1e-6)(inputs + dropout_1)
        #Use the rate variable for the dropout layers and remember to use two dense layers
        # Feed-Forward Layer
        dense_1 = layers.Dense(units=self.ff_dim, activation='relu')(normlayer_1)
        # Second Dense layer
        dense_2 = layers.Dense(units=self.ff_dim)(dense_1)
        # Second Dropout layer
        dropout_2 = layers.Dropout(rate=self.rate)(dense_2)
        # Output Layer norm
        transformer_output = layers.LayerNormalization(epsilon=1e-6)(normlayer_1 + dropout_2)
        #See assignment and its figures for more details.
        return transformer_output

    
    def EmbeddingLayer(self, inputs):
        #create the embedding layer
        #create (1) an embedding for the tokens and (2) an embedding for the positions
        #you can use https://keras.io/api/layers/core_layers/embedding/ Embedding class
        #you can use tf.range to enocde positions
        #add (1) and (2) and return the layer
        embedding_tokens = layers.Embedding(input_dim=self.vocab_size, output_dim=self.embed_dim)(inputs)
        embedding_positions = layers.Embedding(input_dim=self.maxlen, output_dim=self.embed_dim)(tf.range(start=0, limit=self.maxlen, delta=1))
        embeddings = embedding_tokens + embedding_positions
        return embeddings

    
    def create_model(self):
        #combine the EmbeddingLayer and num_blocks TransformerBlocks to create the model, use the Keras functional API (https://keras.io/guides/functional_api/)
        #use the SparseCategoricalCrossentropy loss function (https://keras.io/api/losses/probabilistic_losses/#sparsecategoricalcrossentropy-class)
         # Model Input
        input = layers.Input(shape=(self.maxlen,))
        # Embedding Layer
        embeddings = self.EmbeddingLayer(input)
        # Transformer Blocks
        for i in range(self.num_blocks):
            embeddings = self.TransformerBlock(embeddings)
        # Output Layer
        model_output = layers.Dense(units=self.vocab_size, activation='softmax')(embeddings)
        # Create Model
        model = tf.keras.Model(inputs=input, outputs=model_output)
        # Compile Model
        model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
        return model

## Task 2

In [11]:
class DataSet():
    def __init__(self, filename, len):
        #load the text from the file
        with open(filename, 'r') as f:
            self.text = f.read()
        self.len = len
        self.filename = filename

    def prep_text(self):
        #remove all punctuation, set to lowercase, remove duplicate spaces and other whitespace (keep newlines)
        self.text = re.sub(r'[^\w\s]', '', self.text).lower()
        # self.text = re.sub(r'\s+', ' ', self.text)
        self.text = re.sub(r'[  ]', ' ', self.text)
        
        
    def tokenize_text(self):
        #seperate into words, create a vocab and convert the text to a list of numbers using the vocab such that each unique word is represented by its own number number
        self.words = self.text.split(' ')
        self.vocab = np.unique(self.words)
        self.word2idx = {word: i for i, word in enumerate(self.vocab)}
        self.idx2word = {i: word for i, word in enumerate(self.vocab)}
        self.tokens = np.array([self.word2idx[word] for word in self.words])
        

    def create_dataset(self):
        #split the tokenized data into sequences of length len, return the sequences and vocab
        self.prep_text()
        self.tokenize_text()
        
        x, y = [], []
        for i in range(0, len(self.tokens) - self.len, self.len):
            seq_x = self.tokens[i:i+self.len]
            seq_y = self.tokens[i+1:i+self.len+1]
            x.append(seq_x)
            y.append(seq_y)
        self.x = np.array(x)
        self.y = np.array(y)
        
        return self.x, self.y, self.vocab

## Task 3

In [12]:
class GenerateText():
    def __init__(self, model, vocab):
        self.model = model
        self.vocab = {word: i for i, word in enumerate(vocab)}
        self.idx_to_word = {i: word for i, word in enumerate(self.vocab)}

    def generate_text(self, start_string, num_generate=100):
        input_eval = [self.vocab[word] for word in start_string.split(' ') if word in self.vocab]
        input_eval = tf.expand_dims(input_eval, 0)
        text_generated = []
        
        # Here, batch size == 1
        self.model.reset_states()
        for i in range(num_generate):
            predictions = self.model(input_eval)
            # Remove the batch dimension
            predictions = tf.squeeze(predictions, 0)
            predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
            input_eval = tf.expand_dims([predicted_id], 0)
            text_generated.append(self.idx_to_word[predicted_id])

        return ' '.join(text_generated)
    
    def generate_random_text(self, num_generate=100):
        start_string = np.random.choice(list(self.vocab.keys()))
        return self.generate_text(start_string, num_generate)

## Task 4: Model Traning and Testing

In [13]:
#Train the model while periodically generating text to show progress
def train_model(model, vocab, x, y, epochs=50):
    for i in range(epochs):
        model.fit(x, y, epochs=1, batch_size=64, verbose=1)
        print("Epoch:", i + 1)
        if((i+1)%int(epochs)==0):
            # Generate text after each epoch
            generate_text = GenerateText(model, vocab)
            print(generate_text.generate_text("love", num_generate=50))
    
    return model

In [14]:
ds = DataSet('beatles.txt', len=80)
x, y, vocab = ds.create_dataset()
model = TransformerModel(vocab_size=len(vocab), embed_dim=256, num_heads=2, maxlen=80, num_blocks=2, ff_dim=256, rate=0.1).create_model()
model = train_model(model, vocab, x, y, epochs=1)

6/6 [==============================] - 14s 243ms/step - loss: 8.1555 - accuracy: 0.0277
Epoch: 1
tonight
shes bent chance dose minute somebody
who vain
so beat kissin man
how oh
in eye
crabalocker go
when whisper collapsed home
get ease
everyone you
think weh on
a lagoon
didnt son
mr yeah
ive dark
everyone you
tomorrow oh
closer
let oh
well boy
four see
all years
sgt love
but day
good gone
cause paper komm portrait somebody
help alone
i do
shes moonlight
you that poop
from are
sexy rain
bullfrog
doing seen
youre next ooh
my way act down
out


In [15]:
ds = DataSet('beatles.txt', len=80)
x, y, vocab = ds.create_dataset()
model = TransformerModel(vocab_size=len(vocab), embed_dim=256, num_heads=2, maxlen=80, num_blocks=2, ff_dim=256, rate=0.1).create_model()
model = train_model(model, vocab, x, y, epochs=50)

6/6 [==============================] - 8s 240ms/step - loss: 8.1259 - accuracy: 0.0262
Epoch: 1
6/6 [==============================] - 1s 197ms/step - loss: 7.0429 - accuracy: 0.0404
Epoch: 2
6/6 [==============================] - 1s 240ms/step - loss: 6.6510 - accuracy: 0.0347
Epoch: 3
6/6 [==============================] - 1s 238ms/step - loss: 6.6117 - accuracy: 0.0408
Epoch: 4
6/6 [==============================] - 1s 247ms/step - loss: 6.4476 - accuracy: 0.0687
Epoch: 5
6/6 [==============================] - 1s 237ms/step - loss: 6.1432 - accuracy: 0.1001
Epoch: 6
6/6 [==============================] - 2s 362ms/step - loss: 5.8427 - accuracy: 0.1261
Epoch: 7
6/6 [==============================] - 3s 437ms/step - loss: 5.4788 - accuracy: 0.1541
Epoch: 8
6/6 [==============================] - 3s 454ms/step - loss: 5.0880 - accuracy: 0.1988
Epoch: 9
6/6 [==============================] - 1s 173ms/step - loss: 4.6973 - accuracy: 0.2641
Epoch: 10
6/6 [==============================] - 

In [16]:
ds = DataSet('beatles.txt', len=80)
x, y, vocab = ds.create_dataset()
model = TransformerModel(vocab_size=len(vocab), embed_dim=256, num_heads=2, maxlen=80, num_blocks=2, ff_dim=256, rate=0.1).create_model()
model = train_model(model, vocab, x, y, epochs=100)

6/6 [==============================] - 9s 207ms/step - loss: 8.1247 - accuracy: 0.0229
Epoch: 1
6/6 [==============================] - 1s 229ms/step - loss: 7.0420 - accuracy: 0.0384
Epoch: 2
6/6 [==============================] - 2s 302ms/step - loss: 6.6451 - accuracy: 0.0391
Epoch: 3
6/6 [==============================] - 1s 199ms/step - loss: 6.6066 - accuracy: 0.0423
Epoch: 4
6/6 [==============================] - 1s 221ms/step - loss: 6.4073 - accuracy: 0.0792
Epoch: 5
6/6 [==============================] - 1s 206ms/step - loss: 6.1074 - accuracy: 0.1068
Epoch: 6
6/6 [==============================] - 1s 179ms/step - loss: 5.7962 - accuracy: 0.1347
Epoch: 7
6/6 [==============================] - 1s 158ms/step - loss: 5.4274 - accuracy: 0.1570
Epoch: 8
6/6 [==============================] - 1s 208ms/step - loss: 5.0320 - accuracy: 0.2052
Epoch: 9
6/6 [==============================] - 1s 248ms/step - loss: 4.6416 - accuracy: 0.2697
Epoch: 10
6/6 [==============================] - 

In [19]:
ds = DataSet('beatles.txt', len=100)
x, y, vocab = ds.create_dataset()
model = TransformerModel(vocab_size=len(vocab), embed_dim=256, num_heads=10, maxlen=100, num_blocks=10, ff_dim=256, rate=0.1).create_model()
model = train_model(model, vocab, x, y, epochs=1000)

5/5 [==============================] - 35s 540ms/step - loss: 8.2007 - accuracy: 0.0213
Epoch: 1
5/5 [==============================] - 3s 500ms/step - loss: 7.2950 - accuracy: 0.0270
Epoch: 2
5/5 [==============================] - 3s 502ms/step - loss: 6.7860 - accuracy: 0.0271
Epoch: 3
5/5 [==============================] - 3s 501ms/step - loss: 6.6379 - accuracy: 0.0369
Epoch: 4
5/5 [==============================] - 3s 509ms/step - loss: 6.6375 - accuracy: 0.0390
Epoch: 5
5/5 [==============================] - 3s 504ms/step - loss: 6.6264 - accuracy: 0.0356
Epoch: 6
5/5 [==============================] - 3s 507ms/step - loss: 6.6182 - accuracy: 0.0365
Epoch: 7
5/5 [==============================] - 3s 508ms/step - loss: 6.6161 - accuracy: 0.0383
Epoch: 8
5/5 [==============================] - 3s 504ms/step - loss: 6.6145 - accuracy: 0.0388
Epoch: 9
5/5 [==============================] - 3s 507ms/step - loss: 6.6126 - accuracy: 0.0390
Epoch: 10
5/5 [==============================] -


# Report

## Introduction
The goal of this project is to create neural networks which utilize the transformer architecture to create lyrics for a song which might be from the Beatles.  This is done by training the network using the lyrics from all of the previous Beatles songs.  All of the lyrics are input as a list of words which have removed the punctutation, while keeping the newlines.  In terms of song lyrics, it is important to keep the newlines to retain the structure of a song.
Once a desired length of words is input into the model, it is first embedded using both token and positional embedding.  The token embedding allows for each of the words to be given a certain value so that the model can identify what it is and how to output it, similar to a one-hot encoding of all of the words.  The positional embedding is then used to relate where the words are located in the input.  
Following the embedding is the TransformerBlock, which is a multi-headed attention layer followed by simple dense layers.  Attention used the embedding from earlier to find how each word relates to the others in the phrase.  This model allows for the user to choose the number of TransformerBlocks in the model before training, increasing or decreasing the complexity.  After the TransformerBlocks is another dense layer to the outputs, which are also a prechosen length.  
After training, the network can take in a word or phrase to then create an output which would be the potential lyrics to a Beatles like song.  

## Results

## Conclusion

## How to Run Code

Please include any special libraries and list your tf version here.